In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Загрузка датасета из CSV файла
df = pd.read_csv('./healthy_meal_plans.csv')

print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
print(df.head())

Размер датасета: (500, 14)

Первые 5 строк:
              meal_name  num_ingredients  calories  prep_time   protein  \
0     Gluten-Free Pasta         0.272727  0.490909       0.84  0.783679   
1        Grilled Salmon         0.909091  0.158182       0.70  0.143588   
2           Lentil Soup         0.454545  0.700000       0.40  0.620637   
3         Chickpea Stew         0.909091  0.105455       0.82  0.046893   
4  Turkey Lettuce Wraps         0.181818  0.441818       0.04  0.864340   

        fat     carbs  vegan  vegetarian  keto  paleo  gluten_free  \
0  0.597907  0.444572      0           0     0      0            1   
1  0.652082  0.050609      0           0     1      1            1   
2  0.612748  0.000688      0           1     0      0            0   
3  0.975761  0.229026      1           1     0      0            0   
4  0.681575  0.449293      0           0     1      1            1   

   mediterranean  is_healthy  
0              0           1  
1              1      

### Нормализация данных
Нормализация — это процесс приведения числовых признаков к единому масштабу (обычно от 0 до 1).
Это важно для:
- Корректного сравнения признаков с разными единицами измерения
- Более стабильной работы алгоритма
- Ускорения обучения модели

Используем MinMaxScaler для масштабирования числовых признаков в диапазон [0, 1].

In [3]:
# Создаем копию датасета для нормализации
df_normalized = df.copy()

# Выделяем числовые признаки (исключаем название блюда и целевую переменную)
numeric_features = df.drop(columns=['meal_name', 'is_healthy']).columns.tolist()

print("Статистика ДО нормализации:")
print(df[numeric_features].describe())

# Создаем и применяем MinMaxScaler
scaler = MinMaxScaler()
df_normalized[numeric_features] = scaler.fit_transform(df[numeric_features])

print("\n" + "="*70)
print("Статистика ПОСЛЕ нормализации:")
print(df_normalized[numeric_features].describe())

print("\n" + "="*70)
print("Примеры нормализованных значений (первые 5 строк):")
print(df_normalized[numeric_features].head())

# Используем нормализованные данные для дальнейшей работы
df = df_normalized

Статистика ДО нормализации:
       num_ingredients    calories   prep_time     protein         fat  \
count       500.000000  500.000000  500.000000  500.000000  500.000000   
mean          0.489818    0.504149    0.486480    0.505704    0.506822   
std           0.305054    0.290267    0.293234    0.291046    0.282906   
min           0.000000    0.000000    0.000000    0.000000    0.000000   
25%           0.272727    0.231818    0.220000    0.236392    0.257945   
50%           0.454545    0.509091    0.490000    0.521935    0.516733   
75%           0.727273    0.751364    0.740000    0.764155    0.751306   
max           1.000000    1.000000    1.000000    1.000000    1.000000   

            carbs       vegan  vegetarian        keto       paleo  \
count  500.000000  500.000000  500.000000  500.000000  500.000000   
mean     0.507081    0.334000    0.574000    0.366000    0.388000   
std      0.292257    0.472112    0.494989    0.482192    0.487783   
min      0.000000    0.000000

In [4]:
# Реализуем класс узла
class Node:
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        
# И класс терминального узла (листа)
class Leaf:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его
        prediction = max(classes, key=classes.get)
        return prediction

### 1) Реализация функции расчета критерия Джини
Критерий Джини — это метрика, которая измеряет степень неоднородности набора данных. Она показывает, насколько хорошо разделены классы в узле дерева решений.

In [5]:
# Расчет критерия Джини Чем меньше Gini, тем чище узел.
def gini(labels):
    """
    Вычисляет критерий Джини для набора меток.
    Gini = 1 - sum(p_i^2), где p_i - доля объектов класса i
    """
    # Количество объектов
    total = len(labels)
    
    if total == 0:
        return 0
    
    # Подсчитаем количество объектов каждого класса
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    # Рассчитаем критерий Джини
    impurity = 1
    for label in classes:
        prob = classes[label] / total
        impurity -= prob ** 2
    
    return impurity

### 2) Реализация расчета критерия прироста информации
Прирост информации — это метрика, которая показывает, насколько улучшилось разбиение данных после применения определенного признака и порога. Это основной критерий для выбора наилучшего разбиения в деревьях решений.

In [6]:
# Расчет прироста информации
def gain(left_labels, right_labels, root_gini):
    """
    Вычисляет прирост информации от разбиения.
    Gain = Gini(parent) - weighted_average(Gini(left), Gini(right))
    """
    # Вероятность попадания в левое поддерево
    p_left = len(left_labels) / (len(left_labels) + len(right_labels))
    
    # Вероятность попадания в правое поддерево
    p_right = len(right_labels) / (len(left_labels) + len(right_labels))
    
    # Считаем прирост информации
    information_gain = root_gini - (p_left * gini(left_labels) + p_right * gini(right_labels))
    # Gini(parent) — неоднородность родительского узла
    # P(left) — доля объектов, попавших в левое поддерево
    # P(right) — доля объектов, попавших в правое поддерево
    # Gini(left) — неоднородность левого поддерева
    # Gini(right) — неоднородность правого поддерева

    return information_gain

In [7]:
# Разбиение датасета в узле
def split(data, labels, column_index, t):
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)

    true_data = data[left]
    false_data = data[right]

    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

### 3) Реализация критериев останова (не менее двух)
Критерии останова — это правила, которые определяют, когда дерево должно прекратить разбиение узлов и создать лист (терминальный узел). Без них дерево будет расти до тех пор, пока каждый лист не будет содержать один объект, что приведет к переобучению

In [8]:
# Нахождение наилучшего разбиения с критериями останова
def find_best_split(data, labels, min_samples_leaf=1):
    """
    Критерии останова:
    1) min_samples_leaf - минимальное количество объектов в листе
    2) Проверка на пустые разбиения
    """
    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None

    n_features = data.shape[1]

    for index in range(n_features):
        t_values = np.unique(data[:, index])

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            # Критерий останова 1: минимальное количество объектов в листе
            if len(true_labels) < min_samples_leaf or len(false_labels) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)

            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [9]:
# Построение дерева с помощью рекурсивной функции
def build_tree(data, labels, min_samples_leaf=1, max_depth=None, current_depth=0):
    """
    Критерии останова:
    1) gain == 0 - нет прироста информации
    2) max_depth - максимальная глубина дерева
    3) min_samples_leaf - минимальное количество объектов в листе (внутри find_best_split)
    """
    
    # Критерий останова 2: максимальная глубина
    if max_depth is not None and current_depth >= max_depth:
        return Leaf(data, labels)
    
    gain_value, t, index = find_best_split(data, labels, min_samples_leaf)

    # Критерий останова 1: прекращаем рекурсию, когда нет прироста в качестве
    if gain_value == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, min_samples_leaf, max_depth, current_depth + 1)
    false_branch = build_tree(false_data, false_labels, min_samples_leaf, max_depth, current_depth + 1)

    # Возвращаем класс узла со всеми поддеревьями
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):
    # Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)


def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [11]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing="", feature_names=None):
    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    if feature_names is not None:
        feature_name = feature_names[node.index]
        print(spacing + f'{feature_name} (индекс {node.index}) <= {node.t:.4f}')
    else:
        print(spacing + f'Индекс {node.index} <= {node.t:.4f}')

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ", feature_names)

    # Рекурсионный вызов функции на отрицательном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ", feature_names)

### 4) Реализация функции подсчета метрики модели
Метрика модели — это числовая оценка качества работы модели машинного обучения. Она показывает, насколько хорошо модель справляется со своей задачей.

In [12]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    """
    Вычисляет точность (accuracy) модели.
    Accuracy = (количество правильных предсказаний) / (общее количество предсказаний)
    """
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    
    return correct / len(actual)

### 5) Проверка работы самописного дерева и сравнение со sklearn

In [13]:
# Подготовка данных
# Удаляем столбец с названием блюда и целевую переменную
X = df.drop(columns=['meal_name', 'is_healthy']).values
y = df['is_healthy'].values

# Сохраним имена признаков для удобства вывода
feature_names = df.drop(columns=['meal_name', 'is_healthy']).columns.tolist()

print("Размер датасета:", X.shape)
print("Признаки:", feature_names)
print("\nРаспределение классов в целевой переменной:")
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Класс {label}: {count} объектов ({count/len(y)*100:.1f}%)")

Размер датасета: (500, 12)
Признаки: ['num_ingredients', 'calories', 'prep_time', 'protein', 'fat', 'carbs', 'vegan', 'vegetarian', 'keto', 'paleo', 'gluten_free', 'mediterranean']

Распределение классов в целевой переменной:
  Класс 0: 453 объектов (90.6%)
  Класс 1: 47 объектов (9.4%)


In [14]:
# Построение самописного дерева
print("=" * 70)
print("САМОПИСНОЕ ДЕРЕВО РЕШЕНИЙ")
print("=" * 70)

my_tree = build_tree(X, y, min_samples_leaf=1, max_depth=10)

print("\nСтруктура дерева:")
print_tree(my_tree, feature_names=feature_names)

# Предсказания
my_predictions = predict(X, my_tree)
print("\n" + "="*70)
print("Первые 10 предсказаний самописного дерева:", my_predictions[:10])
print("Первые 10 истинных значений:              ", list(y[:10]))

# Точность
my_accuracy = accuracy_metric(y, my_predictions)
print(f"\nТочность самописного дерева: {my_accuracy:.4f} ({my_accuracy*100:.2f}%)")

САМОПИСНОЕ ДЕРЕВО РЕШЕНИЙ

Структура дерева:
fat (индекс 4) <= 0.6639
--> True:
  protein (индекс 3) <= 0.2940
  --> True:
    Прогноз: 0
  --> False:
    calories (индекс 1) <= 0.7273
    --> True:
      calories (индекс 1) <= 0.2055
      --> True:
        prep_time (индекс 2) <= 0.9200
        --> True:
          Прогноз: 0
        --> False:
          num_ingredients (индекс 0) <= 0.0909
          --> True:
            Прогноз: 0
          --> False:
            Прогноз: 1
      --> False:
        carbs (индекс 5) <= 0.7139
        --> True:
          protein (индекс 3) <= 0.8258
          --> True:
            num_ingredients (индекс 0) <= 0.8182
            --> True:
              num_ingredients (индекс 0) <= 0.0909
              --> True:
                Прогноз: 0
              --> False:
                carbs (индекс 5) <= 0.0313
                --> True:
                  Прогноз: 0
                --> False:
                  Прогноз: 1
            --> False:
              

In [15]:
# Построение дерева из sklearn
print("\n" + "=" * 70)
print("ДЕРЕВО РЕШЕНИЙ ИЗ SKLEARN")
print("=" * 70)

sklearn_tree = DecisionTreeClassifier(random_state=42, min_samples_leaf=1)
sklearn_tree.fit(X, y)

# Предсказания sklearn
sklearn_predictions = sklearn_tree.predict(X)
print("\nПервые 10 предсказаний sklearn дерева:    ", list(sklearn_predictions[:10]))
print("Первые 10 истинных значений:              ", list(y[:10]))

# Точность sklearn
sklearn_accuracy = accuracy_score(y, sklearn_predictions)
print(f"\nТочность sklearn дерева: {sklearn_accuracy:.4f} ({sklearn_accuracy*100:.2f}%)")


ДЕРЕВО РЕШЕНИЙ ИЗ SKLEARN

Первые 10 предсказаний sklearn дерева:     [np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0)]
Первые 10 истинных значений:               [np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0)]

Точность sklearn дерева: 1.0000 (100.00%)


In [16]:
# Сравнение результатов
print("\n" + "=" * 70)
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("=" * 70)

print(f"\nТочность самописного дерева: {my_accuracy:.4f}")
print(f"Точность sklearn дерева:     {sklearn_accuracy:.4f}")
print(f"Разница в точности:          {abs(my_accuracy - sklearn_accuracy):.4f}")

# Сравнение предсказаний
match = np.array(my_predictions) == sklearn_predictions
print(f"\nСовпадение предсказаний: {np.sum(match)}/{len(y)} ({np.sum(match)/len(y)*100:.2f}%)")

if np.all(match):
    print("\nРезультаты совпали!")
else:
    print("\nРезультаты отличаются")
    diff_count = 0
    for i in range(len(y)):
        if my_predictions[i] != sklearn_predictions[i]:
            print(f"  Индекс {i}: Самописное={my_predictions[i]}, sklearn={sklearn_predictions[i]}, Истинное={y[i]}")
            diff_count += 1


СРАВНЕНИЕ РЕЗУЛЬТАТОВ

Точность самописного дерева: 1.0000
Точность sklearn дерева:     1.0000
Разница в точности:          0.0000

Совпадение предсказаний: 500/500 (100.00%)

Результаты совпали!
